# ETL: OFERTA AUTOBUSES ENTRE LOS AÑOS 2019 - 2021

In [1]:
import pandas as pd
import numpy as np
import glob

# Esta librería puede fallar si se ejecuta en Google Colabs y similares.
import locale
locale.setlocale(locale.LC_TIME, "es_ES") 

'es_ES'

## Carga de los ficheros

In [ ]:
files = glob.glob("../datos/Coches_Cuadro_Oferta_Real/*/Coches_Cuadro_Oferta_Real_*.csv")

# Read column names from file
cols = list(pd.read_csv(files[0], sep=';',nrows=1))
cols = list(filter(lambda col: col.strip(), cols))

#Cargamos los datos y especificamos los tipos en las columnas, así como eliminar la última columna por ser vacia
dfs = [pd.read_csv(f, header=0, sep=";",encoding = "ISO-8859-1",dtype={'Elinea':'str'},usecols=cols) for f in files]
df = pd.concat(dfs,ignore_index=True)


# Eliminar los datos en memeria de  los ficheros
del dfs

# Exportamos el dataframe resultante (Datos brutos, ficheros unificados).
df.to_csv('../datos/Coches_Cuadro_Oferta_Real/autobuses_oferta_real.csv')

## Creación de nuevas columnas e información

In [2]:
oferta = pd.read_csv('../datos/Coches_Cuadro_Oferta_Real/autobuses_oferta_real.csv',dtype={'Elinea':'str'})
oferta.drop(columns='Unnamed: 0',inplace=True)
oferta.shape

(3866436, 7)

1. Columna `FServicio` en formato fecha.
2. Nueva columna para saber si esa fecha era un día de lunes a viernes (True) o sábado y domingo (False)
3. Completamos la columna `Coches` con 0 si alguna fila no tiene valor.

In [3]:
oferta['FServicio'] = pd.to_datetime(oferta['FServicio'], format='%d/%m/%Y')
oferta.loc[oferta['FServicio'].dt.dayofweek > 4,'Diario'] = False
oferta['Diario'].fillna(True,inplace=True)
oferta['Coches'].fillna(0,inplace=True)

Se opta por la creación de una tabla auxiliar ya que reduce el tiempo de ejecución.
Con la tabla auxiliar es necesario solo realizar un join entre índices, reduciendo así el computo.

La otra forma era realizando la siguiente operación:

```python
%%time
oferta['fechaCorta'] = oferta['FServicio'].dt.strftime('%B.%Y')
```
| |Tiempo|
|-|-|
|CPU times: total:| 33.6 s|
|Wall time: |38.8 s|

In [4]:
%%time
tiempos = pd.DataFrame({'FServicio': pd.date_range('2019-01-01', '2021-12-31', freq='D')})
tiempos['fechaCorta'] = tiempos['FServicio'].dt.strftime('%B.%Y')
if 'fechaCorta' not in oferta.columns:
    oferta = oferta.join(tiempos.set_index('FServicio'),on='FServicio')

CPU times: total: 391 ms
Wall time: 505 ms


In [5]:
# https://stackoverflow.com/questions/38869778/pandas-set-column-equal-to-grouped-sum-of-another-column?noredirect=1&lq=1

oferta['MediaCochesMes'] = oferta.groupby(by=['CLinea', 'fechaCorta', 'IDFranja']).Coches.transform('mean')

### Calendar Dates GFTS
Este archivo contiene fechas en los que el servicio ha cambiado `Laborable` a `Festivo` excepcionalmente.

In [6]:
calendar_dates = pd.read_csv('../datos/GFTS/gfts_calendar_dates.csv')
calendar_dates['date_formated'] = pd.to_datetime(calendar_dates['date_formated'])
calendar_festivo = calendar_dates.loc[calendar_dates['exception_type']==1,['exception_type','date_formated']]

¿Por qué al hacer merge pasamos de tener 3.5M de filas a tener 7.5M?

[Explicación de StackOverflow](https://stackoverflow.com/a/39019766)

In [7]:
# Unimos los dataframe de oferta y calendar_festivo para cambiar a Diario=False
# las fechas que aparezcan en este segundo dataframe.

# oferta = oferta.merge(right=calendar_festivo,\
#                 right_on='date_formated',\
#                 left_on='FServicio',\
#                 how='left'\
#             ).drop(columns='date_formated')

oferta = oferta.join(other=calendar_festivo.set_index('date_formated'), on='FServicio')
oferta['Diario'] = oferta['exception_type']!=1
oferta.drop_duplicates(inplace=True)
oferta.drop(columns=['exception_type'],inplace=True)

In [8]:
oferta.reset_index(drop=True, inplace=True)

In [9]:
medias = oferta[oferta['Diario']==True].copy()

In [16]:
oferta['MediaCochesMes_Laborables'] = oferta[oferta['Diario']==True].groupby(by=['CLinea','IDFranja','fechaCorta']).Coches.transform('mean')

¿Por qué en algunos meses la MediaCochesMes es igual que la MediaCochesMes_Laborables?

In [18]:
oferta[oferta['MediaCochesMes_Laborables']!=oferta['MediaCochesMes']].head(2)

,CLinea,Elinea,Denominacion,FServicio,IDFranja,Intervalo,Coches,Diario,fechaCorta,MediaCochesMes,MediaCochesMes_Laborables
1696,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-04-01,H07,070000 - 075959,7.0,True,abril.2019,5.5,5.678571
1697,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-04-01,H08,080000 - 085959,8.0,True,abril.2019,6.5,6.678571


## Hora punta Madrid
Según el informe anual de [2018 de CRTM](https://www.crtm.es/media/712934/edm18_sintesis.pdf#page=54) y la [Encuesta Sintética de Movilidad - ESM14](https://www.crtm.es/media/519661/esm_2014.pdf) las franjas horarias quedan de la siguiente manera:
| Afluencia  | Intervalo horas | Tipo       | Información                                                    |
| ---------  | --------------- | ---------- | -------------------------------------------------------------- |
| Máxima     | 07:00 - 09:00   | Hora punta | Movilidad ocupacional, trabajo y estudios.                     |
| Baja       | 10:00 - 13:00   | Hora valle |                                                                |
| Media      | 13:00 - 15:00   | Hora punta | Movilidad ocupacional, trabajo y estudios.                     |
| Baja       | 15:30 - 16:00   | Hora valle |                                                                |
| Media      | 16:00 - 18:00   | Hora punta | Salida de los centros escolares o el fin de la jornada laboral.|
| Media-Baja | 18:00 - 23:00   | Decreciente| Salida de los centros escolares o el fin de la jornada laboral.|


_Los viajes por movilidad ocupacional, trabajo y estudios, se producen en dos periodos concretos, entre las 6h y las 9h de la mañana y entre la 13h y las 15 h de la tarde. Existe otro pico entre las 16h y las 18h relacionado con la salida de los centros escolares o el fin de la jornada laboral._


![Distribución horaria de los viajeros según su actividad](../markdown_images/distribución_horaria_viajeros_actividad.png)

In [33]:
horas = ['H07','H08','H09']

horapunta = oferta[(oferta['IDFranja'].isin(horas))].copy()

In [34]:
horapunta.head(5)

,CLinea,Elinea,Denominacion,FServicio,IDFranja,Intervalo,Coches,Diario,fechaCorta,MediaCochesMes,MediaCochesMes_Laborables
0,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H07,070000 - 075959,2.0,True,enero.2019,5.580645,5.580645
1,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H08,080000 - 085959,3.0,True,enero.2019,6.580645,6.580645
2,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H09,090000 - 095959,3.0,True,enero.2019,7.387097,7.387097
19,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-02,H07,070000 - 075959,6.0,True,enero.2019,5.580645,5.580645
20,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-02,H08,080000 - 085959,7.0,True,enero.2019,6.580645,6.580645


In [35]:
horapunta['MediaCochesDía_HoraPunta'] = horapunta.groupby(by=['CLinea', 'FServicio'],as_index=False)['Coches'].transform('mean')

In [44]:
horapunta['MediaCochesMes'] = horapunta.groupby(by=['CLinea', 'fechaCorta'],as_index=False)['Coches'].transform('mean')

horapunta['MediaCochesMes_Laborables'] = horapunta[horapunta['Diario']==True].groupby(by=['CLinea', 'fechaCorta'],as_index=False)['Coches'].transform('mean')

In [45]:
# Cuando concateno los dos conjuntos de datos tengo que crear una nueva fila
# para la IDFranaja => 'HoraPunta' que tenga como número de Coches => 'MediaCochesDía_HoraPunta' 

pd.concat([oferta, horapunta.drop_duplicates(subset=['CLinea','FServicio']).sort_values(by=['CLinea','FServicio']).reset_index(drop=True).assign(IDFranja='HoraPunta',Coches=horapunta['MediaCochesDía_HoraPunta'])]).sort_values(by=['CLinea','FServicio','IDFranja']).reset_index(drop=True).head(30)

,CLinea,Elinea,Denominacion,FServicio,IDFranja,Intervalo,Coches,Diario,fechaCorta,MediaCochesMes,MediaCochesMes_Laborables,MediaCochesDía_HoraPunta,MediaCochesMes_HoraPunta
0,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H07,070000 - 075959,2.000000,True,enero.2019,5.580645,5.580645,NaN,NaN
1,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H08,080000 - 085959,3.000000,True,enero.2019,6.580645,6.580645,NaN,NaN
2,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H09,090000 - 095959,3.000000,True,enero.2019,7.387097,7.387097,NaN,NaN
3,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H10,100000 - 105959,4.000000,True,enero.2019,8.903226,8.903226,NaN,NaN
4,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H11,110000 - 115959,4.000000,True,enero.2019,9.193548,9.193548,NaN,NaN
5,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H12,120000 - 125959,4.000000,True,enero.2019,9.193548,9.193548,NaN,NaN
6,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H13,130000 - 135959,4.000000,True,enero.2019,9.193548,9.193548,NaN,NaN
7,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H14,140000 - 145959,4.000000,True,enero.2019,9.290323,9.290323,NaN,NaN
8,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H15,150000 - 155959,4.000000,True,enero.2019,8.451613,8.451613,NaN,NaN
9,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H16,160000 - 165959,4.000000,True,enero.2019,8.451613,8.451613,NaN,NaN


In [ ]:
oferta = pd.concat([oferta, horapunta.drop_duplicates(subset=['CLinea','FServicio']).sort_values(by=['CLinea','FServicio']).reset_index(drop=True).assign(IDFranja='HoraPunta')]).sort_values(by=['CLinea','FServicio','IDFranja']).reset_index(drop=True)

# del horapunta